In [2]:
import bs4 as bs
import pickle
import requests

In [28]:
def save_sp500_tickers():
#     resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    resp = requests.get('http://en.wikipedia.org/wiki/S%26P_100')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

the_list = save_sp500_tickers()

In [29]:
the_list = [item[:-1] for item in the_list]
the_list

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK.B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WBA',
 'WFC',
 'WMT',
 'XOM']

In [30]:
general_information = the_list[:]
general_information = [item.replace('.','-') for item in the_list]
general_information

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WBA',
 'WFC',
 'WMT',
 'XOM']

In [3]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
sns.set_style('whitegrid')
from os import listdir
from os.path import isfile, join
from pandas_datareader import data
import yfinance as yf
from pandas_datareader import wb

import quandl
import pandas_datareader as pdr
import numpy as np

from datetime import date

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import copy
from tqdm import tqdm

import math
# import pmdarima as pm
yf.pdr_override()

In [32]:
stock_name = 'AAPL'
start_date = "2015-01-01"
end_date = "2022-12-31"
to_pred_stock_df = data.get_data_yahoo(tickers=stock_name, start = start_date, end = end_date)

# to_pred_stock_df = to_pred_stock_df.drop(to_pred_stock_df[to_pred_stock_df.index<start_date].index)
to_pred_stock_df.reset_index(inplace=True)

for index, the_name in enumerate(general_information):
    stock_df = data.get_data_yahoo(tickers=the_name, start=start_date, end=end_date)
    stock_df.reset_index(inplace=True)
    print(the_name)
    print(stock_df.head())
    if index == 0:
        general_df = to_pred_stock_df[["Date", "Close"]]
        general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name)+"_Close"}), on="Date")
    else:
        general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name)+"_Close"}), on="Date")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AAPL
        Date       Open       High        Low      Close  Adj Close     Volume
0 2014-12-31  28.205000  28.282499  27.552500  27.594999  24.801622  165613600
1 2015-01-02  27.847500  27.860001  26.837500  27.332500  24.565695  212818400
2 2015-01-05  27.072500  27.162500  26.352501  26.562500  23.873642  257142000
3 2015-01-06  26.635000  26.857500  26.157499  26.565001  23.875889  263188400
4 2015-01-07  26.799999  27.049999  26.674999  26.937500  24.210684  160423600
[*********************100%***********************]  1 of 1 completed
ABBV
        Date       Open       High        Low      Close  Adj Close   Volume
0 2014-12-31  66.720001  67.099998  65.349998  65.440002  46.219624  3917300
1 2015-01-02  65.440002  66.400002  65.440002  65.889999  46.537453  5086100
2 2015-01-05  65.500000  66.099998  63.880001  64.650002  45.661648  8007400
3

[*********************100%***********************]  1 of 1 completed
BKNG
        Date         Open         High          Low        Close    Adj Close  \
0 2014-12-31  1150.930054  1154.000000  1140.209961  1140.209961  1140.209961   
1 2015-01-02  1144.000000  1149.439941  1131.510010  1142.060059  1142.060059   
2 2015-01-05  1138.369995  1138.369995  1090.739990  1097.579956  1097.579956   
3 2015-01-06  1103.410034  1103.979980  1072.500000  1079.959961  1079.959961   
4 2015-01-07  1090.770020  1092.069946  1058.569946  1069.569946  1069.569946   

    Volume  
0   482500  
1   509300  
2  1051300  
3  1081500  
4   955000  
[*********************100%***********************]  1 of 1 completed
BLK
        Date        Open        High         Low       Close   Adj Close  \
0 2014-12-31  362.339996  362.959991  356.910004  357.559998  290.133148   
1 2015-01-02  357.980011  359.890015  352.440002  356.730011  289.459625   
2 2015-01-05  353.239990  354.269989  346.600006  347.500000

[*********************100%***********************]  1 of 1 completed
CVX
        Date        Open        High         Low       Close  Adj Close  \
0 2014-12-31  111.650002  113.309998  111.529999  112.180000  78.661652   
1 2015-01-02  111.629997  113.000000  110.849998  112.580002  78.942146   
2 2015-01-05  110.959999  111.199997  107.440002  108.080002  75.786697   
3 2015-01-06  107.870003  109.019997  106.480003  108.029999  75.751640   
4 2015-01-07  109.250000  109.730003  107.510002  107.940002  75.688530   

     Volume  
0   6411800  
1   5898800  
2  11758100  
3  11591600  
4  10353800  
[*********************100%***********************]  1 of 1 completed
DHR
        Date       Open       High        Low      Close  Adj Close   Volume
0 2014-12-31  65.625473  65.822594  64.958305  64.981049  62.350437  2734683
1 2015-01-02  65.208488  65.420776  64.526154  64.988625  62.357727  2722152
2 2015-01-05  64.662621  64.897652  64.025780  64.344200  61.739361  3985095
3 2015-01-0

[*********************100%***********************]  1 of 1 completed
GS
        Date        Open        High         Low       Close   Adj Close  \
0 2014-12-31  196.649994  196.830002  193.770004  193.830002  166.912003   
1 2015-01-02  195.300003  195.729996  192.699997  194.410004  167.411438   
2 2015-01-05  193.059998  194.039993  187.479996  188.339996  162.184418   
3 2015-01-06  188.300003  188.660004  183.929993  184.529999  158.903549   
4 2015-01-07  186.850006  187.990005  185.770004  187.279999  161.271637   

    Volume  
0  1566800  
1  1877700  
2  3413200  
3  3429200  
4  1896800  
[*********************100%***********************]  1 of 1 completed
HD
        Date        Open        High         Low       Close  Adj Close  \
0 2014-12-31  104.750000  106.019997  104.559998  104.970001  86.889282   
1 2015-01-02  105.160004  105.660004  102.669998  103.430000  85.614525   
2 2015-01-05  102.779999  102.779999  100.790001  101.260002  83.818298   
3 2015-01-06  102.070

[*********************100%***********************]  1 of 1 completed
MDLZ
        Date       Open       High        Low      Close  Adj Close    Volume
0 2014-12-31  37.029999  37.139999  36.310001  36.330002  30.852160   5024200
1 2015-01-02  36.430000  36.939999  36.270000  36.790001  31.242805   7707700
2 2015-01-05  36.709999  36.770000  36.119999  36.240002  30.775728   9621100
3 2015-01-06  36.369999  36.650002  35.790001  35.990002  30.563414   7669500
4 2015-01-07  36.189999  37.200001  35.970001  37.119999  31.523041  10418800
[*********************100%***********************]  1 of 1 completed
MDT
        Date       Open       High        Low      Close  Adj Close    Volume
0 2014-12-31  73.279999  73.750000  72.110001  72.199997  60.060631   4737400
1 2015-01-02  72.129997  72.989998  71.410004  71.879997  59.794422   6029800
2 2015-01-05  71.500000  71.839996  71.089996  71.580002  59.544853  11258700
3 2015-01-06  72.000000  72.470001  70.870003  71.220001  59.245399  1283

[*********************100%***********************]  1 of 1 completed
PG
        Date       Open       High        Low      Close  Adj Close   Volume
0 2014-12-31  92.040001  92.550003  91.029999  91.089996  71.816963  5712100
1 2015-01-02  90.839996  91.000000  89.919998  90.440002  71.304497  7251400
2 2015-01-05  90.230003  91.000000  89.849998  90.010002  70.965492  8626100
3 2015-01-06  90.309998  90.559998  89.260002  89.599998  70.642204  7791200
4 2015-01-07  89.940002  90.370003  89.559998  90.070000  71.012779  5986600
[*********************100%***********************]  1 of 1 completed
PM
        Date       Open       High        Low      Close  Adj Close    Volume
0 2014-12-31  82.730003  82.820000  81.360001  81.449997  53.443592   3661300
1 2015-01-02  81.599998  81.790001  80.629997  81.019997  53.161438   3787700
2 2015-01-05  80.989998  81.089996  79.860001  80.410004  52.761200   6213400
3 2015-01-06  80.959999  82.050003  80.599998  81.099998  53.213932  10001900
4 20

[*********************100%***********************]  1 of 1 completed
UNH
        Date        Open        High         Low       Close  Adj Close  \
0 2014-12-31  103.120003  103.680000  101.029999  101.089996  89.196602   
1 2015-01-02  101.500000  102.620003  100.010002  100.779999  88.923096   
2 2015-01-05  100.300003  100.449997   98.750000   99.120003  87.458405   
3 2015-01-06   99.830002  100.139999   98.459999   98.919998  87.281929   
4 2015-01-07   98.959999  100.300003   98.919998   99.930000  88.173111   

    Volume  
0  2313900  
1  3060900  
2  4679000  
3  3468300  
4  3225800  
[*********************100%***********************]  1 of 1 completed
UNP
        Date        Open        High         Low       Close  Adj Close  \
0 2014-12-31  120.720001  120.769997  119.019997  119.129997  99.186287   
1 2015-01-02  119.930000  120.800003  117.660004  118.610001  98.753296   
2 2015-01-05  118.309998  118.309998  114.470001  114.599998  95.414627   
3 2015-01-06  114.279999 

In [33]:
general_df.to_csv('sp500_df.csv')

In [63]:
general_df = pd.read_csv('sp500_df.csv', index_col=0)
general_df

,Date,Close,AAPL_Close,ABBV_Close,ABT_Close,ACN_Close,ADBE_Close,AIG_Close,AMD_Close,AMGN_Close,...,UNH_Close,UNP_Close,UPS_Close,USB_Close,V_Close,VZ_Close,WBA_Close,WFC_Close,WMT_Close,XOM_Close
0,2019-03-20,47.040001,47.040001,80.430000,79.940002,166.270004,259.739990,42.959999,25.700001,191.119995,...,250.500000,161.000000,107.919998,50.369999,153.750000,57.669998,62.790001,50.400002,98.639999,81.320000
1,2019-03-21,48.772499,48.772499,81.239998,80.059998,168.240005,264.179993,43.430000,27.889999,191.889999,...,252.020004,162.610001,109.800003,49.660000,155.800003,58.290001,63.599998,49.860001,99.059998,81.790001
2,2019-03-22,47.762501,47.762501,79.760002,77.970001,165.240005,259.690002,42.619999,26.370001,186.699997,...,247.089996,160.619995,107.349998,48.279999,153.070007,59.759998,62.410000,48.310001,98.279999,80.480003
3,2019-03-25,47.185001,47.185001,79.510002,78.230003,165.080002,262.269989,42.450001,25.969999,186.589996,...,246.860001,161.500000,106.900002,48.099998,153.029999,60.080002,61.689999,48.080002,98.169998,79.919998
4,2019-03-26,46.697498,46.697498,80.760002,79.000000,166.940002,265.589996,43.049999,25.690001,188.270004,...,243.389999,161.809998,108.410004,48.509998,155.300003,60.630001,61.919998,49.009998,98.320000,80.959999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,2022-12-23,131.860001,131.860001,163.100006,108.180000,266.089996,338.450012,63.160000,64.519997,263.920013,...,531.309998,209.910004,177.009995,43.200001,205.830002,38.410000,38.630001,40.980000,143.770004,108.680000
951,2022-12-27,130.029999,130.029999,162.990005,108.570000,265.309998,335.089996,63.330002,63.270000,263.390015,...,531.989990,210.320007,176.929993,43.650002,206.289993,39.250000,38.310001,41.040001,143.809998,110.190002
952,2022-12-28,126.040001,126.040001,162.229996,107.830002,263.119995,328.329987,62.889999,62.570000,261.420013,...,528.450012,206.869995,173.610001,43.349998,204.990005,38.810001,37.580002,41.119999,141.289993,108.379997
953,2022-12-29,129.610001,129.610001,162.559998,110.309998,268.380005,337.579987,63.630001,64.820000,263.160004,...,529.880005,209.220001,175.750000,43.570000,208.059998,39.259998,37.470001,41.330002,142.149994,109.199997


In [64]:
cor_AAPL_df = general_df.drop(columns=['Date', 'Close'])
# the_names= corr_df.columns.tolist()
# x = AAPL_df['Close']
# cor_AAPL_df = AAPL_df[the_names]
# Covariance = np.cov(x, y)
Covariance = cor_AAPL_df.corr()
Covariance = Covariance.round(decimals=12, out=None)

In [65]:
cor_dic = Covariance['AAPL_Close'].to_dict()
sortedList = [k for k, v in sorted(cor_dic.items(), key=lambda item: item[1], reverse = True)]
# sortedDict = sorted(cor_dic.items())

In [66]:
sortedList[0]

'AAPL_Close'

In [67]:
cor_dic[sortedList[0]]

1.0

In [68]:
for the_key1, the_key2 in zip(sortedList[1:51], sortedList[51:]):
#     print(the_key1)
    print(the_key1[:-6], ' & ',  '%.10f' % cor_dic[the_key1] , ' & ', the_key2[:-6], ' & ', '%.10f' % cor_dic[the_key2] , ' \\\\')

TMO  &  0.9749621815  &  PM  &  0.6934372072  \\
MSFT  &  0.9584408917  &  AMZN  &  0.6862779841  \\
DHR  &  0.9575826520  &  DUK  &  0.6835256992  \\
TSLA  &  0.9563119144  &  ADBE  &  0.6792954588  \\
LOW  &  0.9524922058  &  BAC  &  0.6726946972  \\
NEE  &  0.9393401279  &  SO  &  0.6633826895  \\
AVGO  &  0.9365944174  &  HON  &  0.6576001985  \\
LIN  &  0.9346751859  &  COF  &  0.6440110017  \\
COST  &  0.9272982142  &  KO  &  0.6246900536  \\
ACN  &  0.9243416519  &  FDX  &  0.6234879526  \\
UPS  &  0.9230814088  &  AMGN  &  0.5961522488  \\
QCOM  &  0.9196617265  &  GD  &  0.5849544780  \\
NVDA  &  0.9136831784  &  BKNG  &  0.5813012557  \\
UNH  &  0.9070783206  &  AMT  &  0.5681555440  \\
GOOGL  &  0.9052657636  &  CRM  &  0.5436839686  \\
PG  &  0.9051200401  &  JPM  &  0.5414507863  \\
GOOG  &  0.9037163331  &  DOW  &  0.5401116025  \\
MS  &  0.9008001948  &  EXC  &  0.5365882916  \\
AMD  &  0.8998259206  &  GM  &  0.5230489450  \\
UNP  &  0.8988840152  &  COP  &  0.512488761

In [50]:
stock_name = 'AAPL'
start_date = "2015-01-01"
end_date = "2022-12-31"
to_pred_stock_df = data.get_data_yahoo(tickers=stock_name, start = start_date, end = end_date)

# to_pred_stock_df = to_pred_stock_df.drop(to_pred_stock_df[to_pred_stock_df.index<start_date].index)
to_pred_stock_df.reset_index(inplace=True)

for index, the_name in enumerate(sortedList[1:11]):
    stock_df = data.get_data_yahoo(tickers=the_name[:-6], start=start_date, end=end_date)
    stock_df.reset_index(inplace=True)
    print(the_name)
    print(stock_df.head())
    if index == 0:
        general_df = to_pred_stock_df[["Date", "Close"]]
        general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name)}), on="Date")
    else:
        general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name)}), on="Date")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
TMO_Close
        Date        Open        High         Low       Close   Adj Close  \
0 2014-12-31  127.230003  127.839996  125.260002  125.290001  122.190460   
1 2015-01-02  126.300003  128.490005  126.160004  128.119995  124.950439   
2 2015-01-05  127.309998  129.139999  126.150002  126.360001  123.234001   
3 2015-01-06  126.360001  128.190002  124.540001  125.180000  122.083168   
4 2015-01-07  126.559998  128.970001  125.940002  128.929993  125.740402   

    Volume  
0  1045900  
1  2034700  
2  2430300  
3  2519300  
4  2098300  
[*********************100%***********************]  1 of 1 completed
MSFT_Close
        Date       Open       High        Low      Close  Adj Close    Volume
0 2014-12-31  46.730000  47.439999  46.450001  46.450001  40.439579  21552500
1 2015-01-02  46.660000  47.419998  46.540001  46.759998  40.709473  27913900
2 2

In [55]:
general_df.to_csv('top10_sp500_df.csv')

In [56]:
AAPL_df = pd.read_csv('collected_df.csv', index_col=0)
AAPL_df

,Date,Close,Volume,Adj Close,^DJI_Close,^IXIC_Close,^GSPC_Close,^RLG_Close,^NYA_Close,^VIX_Close,...,BACTSAMFRBDAL,IR,IQ,PPIACO,CPIAUCSL,CPILFESL,MICH,CSCICP03USM665S,Grow,News_Score
0,2015-01-02,27.332500,212818400,24.565697,17832.990234,4726.810059,2058.199951,960.280029,10830.919922,17.790001,...,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074,False,0.500000
1,2015-01-05,26.562500,257142000,23.873644,17501.650391,4652.569824,2020.579956,943.780029,10607.679688,19.920000,...,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074,False,0.500375
2,2015-01-06,26.565001,263188400,23.875887,17371.640625,4592.740234,2002.609985,935.020020,10514.870117,21.120001,...,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074,True,0.719541
3,2015-01-07,26.937500,160423600,24.210682,17584.519531,4650.470215,2025.900024,946.880005,10627.129883,19.309999,...,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074,True,0.074792
4,2015-01-08,27.972500,237458000,25.140909,17907.869141,4736.189941,2062.139893,964.159973,10800.540039,17.010000,...,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074,True,0.996247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,2022-12-23,131.860001,63814900,131.658981,33203.929688,10497.860352,3844.820068,2165.620117,15188.450195,20.870001,...,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411,False,0.696560
2003,2022-12-27,130.029999,69007800,129.831772,33241.558594,10353.230469,3829.250000,2144.899902,15218.259766,21.650000,...,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411,False,0.312938
2004,2022-12-28,126.040001,85438400,125.847855,32875.710938,10213.290039,3783.219971,2117.199951,15037.320312,22.139999,...,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411,False,0.175638
2005,2022-12-29,129.610001,75703700,129.412415,33220.800781,10478.089844,3849.280029,2163.790039,15241.209961,21.440001,...,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411,True,0.930547


In [57]:
general_df  = pd.read_csv('top10_sp500_df.csv', index_col=0)
general_df

,Date,Close,TMO_Close,MSFT_Close,DHR_Close,TSLA_Close,LOW_Close,NEE_Close,AVGO_Close,LIN_Close,COST_Close,ACN_Close
0,2014-12-31,27.594999,125.290001,46.450001,64.981049,14.827333,68.800003,26.572500,100.589996,129.559998,141.750000,89.309998
1,2015-01-02,27.332500,128.119995,46.759998,64.988625,14.620667,67.699997,26.672501,100.089996,129.949997,141.610001,88.839996
2,2015-01-05,26.562500,126.360001,46.330002,64.344200,14.006000,66.080002,26.389999,98.489998,126.519997,140.000000,87.339996
3,2015-01-06,26.565001,125.180000,45.650002,63.904472,14.085333,66.230003,26.492500,96.250000,124.900002,141.850006,86.709999
4,2015-01-07,26.937500,128.929993,46.230000,64.291130,14.063333,68.199997,26.735001,98.849998,126.300003,144.320007,88.529999
...,...,...,...,...,...,...,...,...,...,...,...,...
2010,2022-12-23,131.860001,540.679993,238.729996,259.859985,123.150002,201.880005,84.169998,552.429993,329.329987,462.649994,266.089996
2011,2022-12-27,130.029999,545.440002,236.960007,260.880005,109.099998,201.710007,84.000000,553.539978,328.950012,458.500000,265.309998
2012,2022-12-28,126.040001,543.409973,234.529999,259.339996,112.709999,200.059998,83.160004,544.890015,325.910004,452.989990,263.119995
2013,2022-12-29,129.610001,557.010010,241.009995,266.850006,121.820000,202.259995,84.080002,557.809998,329.750000,456.529999,268.380005


In [58]:
AAPL_df = pd.read_csv('collected_df.csv', index_col=0)
general_df  = pd.read_csv('top10_sp500_df.csv', index_col=0)
general_df  = general_df.drop(columns=['Close'])
general_df.reset_index(inplace=True)
AAPL_df = pd.merge(AAPL_df, general_df, on="Date")
AAPL_df

,Date,Close,Volume,Adj Close,^DJI_Close,^IXIC_Close,^GSPC_Close,^RLG_Close,^NYA_Close,^VIX_Close,...,TMO_Close,MSFT_Close,DHR_Close,TSLA_Close,LOW_Close,NEE_Close,AVGO_Close,LIN_Close,COST_Close,ACN_Close
0,2015-01-02,27.332500,212818400,24.565697,17832.990234,4726.810059,2058.199951,960.280029,10830.919922,17.790001,...,128.119995,46.759998,64.988625,14.620667,67.699997,26.672501,100.089996,129.949997,141.610001,88.839996
1,2015-01-05,26.562500,257142000,23.873644,17501.650391,4652.569824,2020.579956,943.780029,10607.679688,19.920000,...,126.360001,46.330002,64.344200,14.006000,66.080002,26.389999,98.489998,126.519997,140.000000,87.339996
2,2015-01-06,26.565001,263188400,23.875887,17371.640625,4592.740234,2002.609985,935.020020,10514.870117,21.120001,...,125.180000,45.650002,63.904472,14.085333,66.230003,26.492500,96.250000,124.900002,141.850006,86.709999
3,2015-01-07,26.937500,160423600,24.210682,17584.519531,4650.470215,2025.900024,946.880005,10627.129883,19.309999,...,128.929993,46.230000,64.291130,14.063333,68.199997,26.735001,98.849998,126.300003,144.320007,88.529999
4,2015-01-08,27.972500,237458000,25.140909,17907.869141,4736.189941,2062.139893,964.159973,10800.540039,17.010000,...,130.270004,47.590000,65.284309,14.041333,69.639999,26.897499,103.790001,128.380005,145.559998,89.879997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,2022-12-23,131.860001,63814900,131.658981,33203.929688,10497.860352,3844.820068,2165.620117,15188.450195,20.870001,...,540.679993,238.729996,259.859985,123.150002,201.880005,84.169998,552.429993,329.329987,462.649994,266.089996
2003,2022-12-27,130.029999,69007800,129.831772,33241.558594,10353.230469,3829.250000,2144.899902,15218.259766,21.650000,...,545.440002,236.960007,260.880005,109.099998,201.710007,84.000000,553.539978,328.950012,458.500000,265.309998
2004,2022-12-28,126.040001,85438400,125.847855,32875.710938,10213.290039,3783.219971,2117.199951,15037.320312,22.139999,...,543.409973,234.529999,259.339996,112.709999,200.059998,83.160004,544.890015,325.910004,452.989990,263.119995
2005,2022-12-29,129.610001,75703700,129.412415,33220.800781,10478.089844,3849.280029,2163.790039,15241.209961,21.440001,...,557.010010,241.009995,266.850006,121.820000,202.259995,84.080002,557.809998,329.750000,456.529999,268.380005


In [59]:
AAPL_df.isnull().values.any()

False

In [60]:
AAPL_df.to_csv('AAPL_Data.csv')

In [52]:
# the_names = general_df.columns.tolist()[2:]
thelist = [item[:-6] for item in sortedList[1:11]]
print(the_names)

['ABBV_Close', 'ABT_Close', 'ACN_Close', 'ADBE_Close', 'AIG_Close', 'AMD_Close', 'AMGN_Close', 'AMT_Close', 'AMZN_Close', 'AVGO_Close']


In [54]:
thelist

['TMO', 'MSFT', 'DHR', 'TSLA', 'LOW', 'NEE', 'AVGO', 'LIN', 'COST', 'ACN']

In [53]:
sortedList

['AAPL_Close',
 'TMO_Close',
 'MSFT_Close',
 'DHR_Close',
 'TSLA_Close',
 'LOW_Close',
 'NEE_Close',
 'AVGO_Close',
 'LIN_Close',
 'COST_Close',
 'ACN_Close',
 'UPS_Close',
 'QCOM_Close',
 'NVDA_Close',
 'UNH_Close',
 'GOOGL_Close',
 'PG_Close',
 'GOOG_Close',
 'MS_Close',
 'AMD_Close',
 'UNP_Close',
 'ABT_Close',
 'TXN_Close',
 'HD_Close',
 'JNJ_Close',
 'TMUS_Close',
 'CVS_Close',
 'ABBV_Close',
 'TGT_Close',
 'BRK-B_Close',
 'MDLZ_Close',
 'SCHW_Close',
 'LLY_Close',
 'GS_Close',
 'WMT_Close',
 'PEP_Close',
 'BLK_Close',
 'MA_Close',
 'CAT_Close',
 'ORCL_Close',
 'MCD_Close',
 'EMR_Close',
 'V_Close',
 'BMY_Close',
 'NKE_Close',
 'F_Close',
 'AXP_Close',
 'KHC_Close',
 'MET_Close',
 'PFE_Close',
 'CL_Close',
 'PM_Close',
 'AMZN_Close',
 'DUK_Close',
 'ADBE_Close',
 'BAC_Close',
 'SO_Close',
 'HON_Close',
 'COF_Close',
 'KO_Close',
 'FDX_Close',
 'AMGN_Close',
 'GD_Close',
 'BKNG_Close',
 'AMT_Close',
 'CRM_Close',
 'JPM_Close',
 'DOW_Close',
 'EXC_Close',
 'GM_Close',
 'COP_Close',
